# Import NASS using Python and Pass to GAMS Directly

The is a quick guide to download data from **USDA NASS** dataset and import to GAMS directly using the newly added GAMS 25 Embedded Python feature. For detailed introduction on the feature see the
__[recent newsletter from Dr. McCarl](https://www.gams.com/fileadmin/community/mccarlarchive/news42.pdf)__.

Last update: Mark 3/8/2018

Before you start, make sure you are using the new **GAMS 25.0.2** version.

## Install Python Module calling NASS API 

USDA have a API to download their dataset. More information about it and to request a **FREE** API for yourself __[here](http://quickstats.nass.usda.gov/api)__.

To use the API, first you need to install the Python module NASS. Here are the steps:

- open windows command prompt
- navigate to folder where GAMSPython and PIP is istalled using <span style="color:red">"cd C:\GAMS\win64\25.0\GMSPython\Scripts"</span>  (make sure you see something like 'pip3.exe' so that pip is installed here)
- use <span style="color:red">'pip install nass'</span> 'pip install nass' to install nass module in GAMSPython

You can use other moduels the same way. For the data cleanups, I also used two other moduels "pandas" and "numpy".


## Importing NASS in GAMS

- import the module used 
- navigate to the working directory 
- set up the API (**Below is the one I requested. Please use link provided above to request you own API for free**)

In [1]:
import nass
import pandas as pd
import numpy as np
import os

os.chdir("YourWorkingPath")

api = nass.NassApi('4A3B90E6-199F-344A-B466-3236FE813C2B')


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'YourWorkingPath'


## Do the query

The basic logic is that, <span style="color:red">"q = api.query()"</span> starts querying process and then you can use the query subjects to filter the database. For example, <span style="color:red">q.filter('source_desc','SURVEY')</span> filtered only the data from SURVEY; q.filter('state_name','TEXAS') filters only information for TEXAS . <span style="color:red">"q.count()"</span> will give you the size of current database. 

In this example, all the filters reduced the database from 3,4659,577 to 49,477.


In [2]:
q = api.query()
print("The total number of NASS dataset is " + str(q.count()))
q.filter('source_desc','SURVEY')
q.filter('sector_desc','ANIMALS & PRODUCTS')
q.filter('group_desc','LIVESTOCK')
q.filter('agg_level_desc','COUNTY')
q.filter('state_name','TEXAS')
print("After filtering we have " + str(q.count()))

NameError: name 'api' is not defined

In [15]:
# Generate the output DataFrame
columns_to_keep = ['year','freq_desc','state_fips_code','county_code','asd_code',
'commodity_desc','prodn_practice_desc','short_desc','unit_desc','Value']
livestock_db_all = pd.DataFrame(q.execute())
livestock_db = livestock_db_all[columns_to_keep]
print(livestock_db.head())

   year      freq_desc state_fips_code county_code asd_code commodity_desc  \
0  2007  POINT IN TIME              48         998       11         CATTLE   
1  2006  POINT IN TIME              48         998       11         CATTLE   
2  2005  POINT IN TIME              48         998       11         CATTLE   
3  2004  POINT IN TIME              48         998       11         CATTLE   
4  2003  POINT IN TIME              48         998       11         CATTLE   

  prodn_practice_desc                   short_desc unit_desc      Value  
0             ON FEED  CATTLE, ON FEED - INVENTORY      HEAD  2,396,000  
1             ON FEED  CATTLE, ON FEED - INVENTORY      HEAD  2,415,000  
2             ON FEED  CATTLE, ON FEED - INVENTORY      HEAD  2,252,000  
3             ON FEED  CATTLE, ON FEED - INVENTORY      HEAD  2,325,000  
4             ON FEED  CATTLE, ON FEED - INVENTORY      HEAD  2,193,000  


### A few things in mind.
- first, NASS only allows you to download less than 50,000 data at one time. So if you want to do download more than that, you have to do it in loops. See the attached example of looping in "part4_cropmix.py".
- second, you can see the name of filters and the options from the html source code at: view-source:https://quickstats.nass.usda.gov/. For example, in line 1114 you can see the NAME for source is <span style="color:red">"source_desc"</span> and the two available options are <span style="color:red"> CENSUS </span> and <span style="color:red"> SURVEY </span>. Similar for others. You can also explore the options for a particular filter with the following python command: api.param_values() see the example below.



		   

In [9]:
## options for a filter
print(api.param_values('source_desc'))

['CENSUS', 'SURVEY']


## Data clean ups 
You need to do some further query and cleanups before get it to GAMS. The following python codes to the following:
- further query to only get data with units as "HEAD"
- rename to short colunm names
- remove missing informations

In [30]:
pd.options.mode.chained_assignment = None
## further query to only get data with units as "HEAD"

set(livestock_db['unit_desc'])
livestock_heads = livestock_db.loc[livestock_db['unit_desc']=='HEAD',]
set(livestock_heads['commodity_desc'])

## rename to short column names
short_desc=list(set(livestock_heads['short_desc']))

conditions = [
 livestock_heads['short_desc']=='CATTLE, COWS, MILK - INVENTORY',
 livestock_heads['short_desc']=='WOOL - SHORN, MEASURED IN HEAD',
 livestock_heads['short_desc']=='CATTLE, ON FEED - PLACEMENTS, MEASURED IN HEAD',

 livestock_heads['short_desc']=='SHEEP, INCL LAMBS - INVENTORY',
 livestock_heads['short_desc']=='GOATS, ANGORA - INVENTORY',
 livestock_heads['short_desc']=='CATTLE, INCL CALVES - INVENTORY',

 livestock_heads['short_desc']=='GOATS, MILK - INVENTORY',
 livestock_heads['short_desc']=='GOATS, MEAT & OTHER - INVENTORY',
 livestock_heads['short_desc']=='SHEEP, EWES, BREEDING, GE 1 YEAR - INVENTORY',

 livestock_heads['short_desc']=='GOATS - INVENTORY',
 livestock_heads['short_desc']=='CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURED IN HEAD',
 livestock_heads['short_desc']=='HOGS - INVENTORY',

 livestock_heads['short_desc']=='MOHAIR, ANGORA - CLIPPED, MEASURED IN HEAD',
 livestock_heads['short_desc']=='CATTLE, ON FEED - INVENTORY',
 livestock_heads['short_desc']=='CATTLE, COWS, BEEF - INVENTORY'
    ]

choices=['cattlecowmilk','sheepwool','cattleonfeed_placement',
        'sheeplambs','goatsangora','cattlecalves',
        'milkgoats','meatgoats','sheepewes',
        'goats','cattleonfeed4slaughter','hogs',
        'angora_mohair','cattleonfeed','beefcow']

livestock_heads['livestock1']=np.select(conditions,choices,default="other")
## the above chunk of code just rename to shorter names for livestocks, if you don't want any to be included, just change the corresponding
## choices to "other" and it would be dropped by the next line of code

## remove missing informations
livestock_heads = livestock_heads.loc[livestock_heads['livestock1'] != 'other']
livestock_heads = livestock_heads.loc[livestock_heads['Value'] != '                 (D)',]
livestock_heads = livestock_heads.loc[livestock_heads['asd_code'] != '99',]
livestock_heads = livestock_heads.loc[livestock_heads['county_code'] != '998',]
livestock_heads['fips']=livestock_heads['state_fips_code']+livestock_heads['county_code']
livestock_export=livestock_heads[['fips','asd_code','livestock1','year','Value']]

livestockmixnew=livestock_export.values.tolist()
for i in range(len(livestockmixnew)):
    try:
        temp=livestockmixnew[i][4].replace(',','')
        int(temp)
    except :
        print(livestockmixnew[i])
        
print("Dataframe ready to export is 'livestock_export'")
print('*' * 50)
print(livestock_export.head())

Dataframe ready to export is 'livestock_export'
**************************************************
     fips asd_code    livestock1  year  Value
94  48011       11  cattleonfeed  1980  3,000
95  48011       11  cattleonfeed  1979  4,000
96  48011       11  cattleonfeed  1978  4,000
97  48011       11  cattleonfeed  1977  4,000
98  48011       11  cattleonfeed  1976  5,000


## Pass to GAMS
Now in the last step we want to pass the generated dataframe **livestock_export** into GAMS. Note it is defined on four domains __fips__, __asd_code__, __livestock1__ and __Value__, we actually need to import four sets and 1 parameter here.  

The GAMS only takes specific forms of python data. Specifically:
- for sets: it only takes <span style="color:red">list</span>
- for parameters: it only takes <span style="color:red">tuples</span> with all domains enter as "**str**" and the value as "**float**" or "**int**".
    
Such conversion is showed as in the code below. Note for the parameter, we need to convert the fifth element (namely x[4]) from **str** to **int** it is the value of the parameter.

In [24]:
livestockmixnew=livestock_export.values.tolist()

## create list for sets
countyfips=list(set(livestock_export['fips']))
usda_dist=list(set(livestock_export['asd_code']))
livestockall=list(set(livestock_export['livestock1']))
mixesa=list(set(livestock_export['year']))

## create tuples for parameters
livestockmixnew=[( x[0],x[1],x[2],x[3],int(x[4].replace(',','')) ) for x in livestockmixnew]

## Run Python in GAMS
now all the above python lines can now be called directly in GAMS with the EmbeddedCode feature. Specifically, you just need to put them between the following two lines.

$onEmbeddedCode Python:

<span style="color:red">ALL the Python Code used so far !!! </span>

$offEmbeddedCode 


## Setups in GAMS
Finally, few steps are needed to complete the whole procedure in the GAMS file below.
- the first line checks if GAMSPython is ready to be used
- 4 empty sets and 1 empty parameters are defined.
- The Embedded Python code we illustrated before
    - With five lines starts with **gams.set** which pass the python data into GAMS
- after the end of the **offEmbeededCode** put the names of imported set and parameters    



In [ ]:
#############################################################################
### This is a GAMS file, Not Python file. So it is NOT runnable here     ####
### Run the Example1_single_import.gms in GAMS instead                   ####
#############################################################################


$if not setenv GMSPYTHONHOME $abort.noError Embedded code Python not ready to be used

set countyfips   /system.empty/
    usda_dist    /system.empty/
    livestockall /system.empty/
    mixesa       /system.empty/
   ;
parameter livestockmixnew(*,usda_dist,livestockall,mixesa) original livestock mix data from usda nass;

$onEmbeddedCode Python:

!!!! ALL the Python Code used so far !!!!!!!

** five additional lines to pass data to GAMS
gams.set("countyfips",countyfips)
gams.set("usda_dist",usda_dist)
gams.set("livestockall",livestockall)
gams.set("mixesa",mixesa)
gams.set("livestockmixnew",livestockmixnew)

$offEmbeddedCode  countyfips usda_dist  livestockall mixesa livestockmixnew
display countyfips, usda_dist, livestockall, mixsa, livestockmixnew;
